In [3]:
'''
https://www.kaggle.com/code/emiz6413/huggingface-deberta-v3-base-finetune

^steal this code, who cares

and/or this code: https://huggingface.co/learn/nlp-course/en/chapter3/4
'''

'\nhttps://www.kaggle.com/code/emiz6413/huggingface-deberta-v3-base-finetune\n\n^steal this code, who cares\n\nand/or this code: https://huggingface.co/learn/nlp-course/en/chapter3/4\n'

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, AdamW, get_scheduler
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

/mnt/c/Users/hew7/documents/venvs/pythia/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'microsoft/deberta-v3-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/mnt/c/Users/hew7/documents/venvs/pythia/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [3]:
raw_data = pd.read_csv('../datasets/biasly/biasly_prepared_df.csv')

In [4]:
len(raw_data)

12697

In [5]:
raw_data.columns

Index(['datapoint', 'misogynistic_label'], dtype='object')

In [6]:
tokenizer("fuck?")

{'input_ids': [1, 25848, 302, 2], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [7]:
#add data via pandas
tokenized_data = pd.DataFrame([tokenizer(x) for x in raw_data["datapoint"]])

In [8]:
total_data = pd.concat((raw_data, tokenized_data), axis=1)

In [9]:
total_data["labels"] = total_data["misogynistic_label"]

In [10]:
class BiaslyDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.texts = dataframe['datapoint'].tolist()
        self.labels = dataframe['misogynistic_label'].tolist()
        
        # Tokenize in the constructor
        self.encodings = tokenizer(self.texts, truncation=True, padding=True, max_length=512)
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [11]:
whole_dataset = BiaslyDataset(raw_data, tokenizer)

# Split Dataset
generator = torch.Generator().manual_seed(42)
train_size = int(0.8 * len(whole_dataset))
val_size = int(0.1 * len(whole_dataset))
test_size = len(whole_dataset) - train_size - val_size

train_set, val_set, test_set = torch.utils.data.random_split(
    whole_dataset, 
    [train_size, val_size, test_size], 
    generator=generator
)

In [ ]:
# ### GARBAGE I COPIED - fix this stuff up
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# tokenized_datasets.set_format("torch")
# tokenized_datasets["train"].column_names

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch_size = 4

train_dataloader = DataLoader(
    train_set, shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    val_set, batch_size=batch_size, collate_fn=data_collator
)


from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

/mnt/c/Users/hew7/documents/venvs/pythia/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


7620


  4%|███▏                                                                          | 314/7620 [02:17<2:46:43,  1.37s/it]